## Texture Descriptors: Local Binary Patterns (LBP)

The base pipeline is given: Extract lbp features + label encoding + minmax scaling + stratified k fold + classifier (linear regression for now)

In [17]:
# Load Libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
import skimage
from scipy.stats import skew, kurtosis
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from skimage.feature import local_binary_pattern
from dataset.dataset import SkinLesion_Dataset, SegExamples
from pipeline.preprocessing import SkinLesionPreprocessing
from sklearn.model_selection import StratifiedKFold
from statistics import stdev
from sklearn import linear_model

preproc = SkinLesionPreprocessing()
data = SegExamples()

In [21]:
# settings for LBP
# Compute the rotation invariant uniform (frequent) LBP features
binary_data = data.seg_examples_df.iloc[:60] # only nevus vs others
paths = "../data/"+binary_data.path

# Create Binary Labels for the sample binary dataset
binary_labels = []
for p in binary_data.path:
     binary_labels.append(p.split('/')[2])
binary_labels = pd.DataFrame(binary_labels)
label_encoder = preprocessing.LabelEncoder()

def lbp_function(paths):
    radius = 1
    n_points = 8 * radius
    lbp_feature_vector=[]
    for i in paths:
        image = cv2.imread(i, cv2.IMREAD_GRAYSCALE) # read image as grayscale
        lbp = local_binary_pattern(image, n_points, radius, method="uniform") # 'uniform' is rotation invariant.
        hist = np.histogram(lbp.ravel())
        feature_array=np.ndarray.tolist(hist[0])
        lbp_feature_vector.append(feature_array)
    return pd.DataFrame(lbp_feature_vector)

In [22]:
df = lbp_function(paths)
df['label'] = label_encoder.fit_transform(binary_labels[0]) # add binary labels
df

,0,1,2,3,4,5,6,7,8,9,label
0,11175,18382,11056,27568,31619,38299,26568,27658,35824,41851,0
1,32752,50577,17370,72636,85644,120208,52069,79234,420069,118017,0
2,84362,95319,41044,62585,58294,77699,76937,119955,197095,235286,0
3,12817,22783,14350,29035,49311,29132,21261,25759,27192,38360,0
4,51041,85057,45705,100709,93264,124879,105947,105429,156252,180293,0
5,30703,60995,36163,128309,136303,181624,105114,93327,147866,128172,0
6,28073,48533,31650,117717,155663,174094,83612,81546,218805,108883,0
7,49790,76545,26934,85139,82337,116537,74716,96165,254324,186089,0
8,31635,61815,36737,123092,183880,170855,99252,83916,127931,129463,0
9,26856,40967,33813,138701,310033,188733,57567,58535,103218,90153,0


In [23]:
def classifier(df):
    x = df.loc[:, df.columns != 'label']
    y = df['label']

    scaler = preprocessing.MinMaxScaler()
    x_scaled = scaler.fit_transform(x)

    # Create  classifier object.
    lr = linear_model.LogisticRegression()
    # lr = svm.SVC()

    # Create StratifiedKFold object.
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)
    lst_accu_stratified = []

    for train_index, test_index in skf.split(x, y):
        x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        lr.fit(x_train_fold, y_train_fold)
        lst_accu_stratified.append(lr.score(x_test_fold, y_test_fold))

    # Print the output.
    print('List of possible accuracy:', lst_accu_stratified)
    print('\nMaximum Accuracy That can be obtained from this model is:',
          max(lst_accu_stratified)*100, '%')
    print('\nMinimum Accuracy:',
          min(lst_accu_stratified)*100, '%')
    print('\nOverall Accuracy:',
          np.mean(lst_accu_stratified)*100, '%')
    print('\nStandard Deviation is:', stdev(lst_accu_stratified))

In [24]:
classifier(df)

List of possible accuracy: [0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666]

Maximum Accuracy That can be obtained from this model is: 66.66666666666666 %

Minimum Accuracy: 66.66666666666666 %

Overall Accuracy: 66.66666666666667 %

Standard Deviation is: 0.0
